# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4941, uptime 0:00:33
mini_beamline                    RUNNING   pid 4942, uptime 0:00:33
random_walk                      RUNNING   pid 4943, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4944, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4945, uptime 0:00:33
simple                           RUNNING   pid 4946, uptime 0:00:33
thermo_sim                       RUNNING   pid 4947, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4948, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 01:24:09
Persistent Unique Scan ID: 'd87feaf3-1509-42a4-91a5-82dd4f7cafa2'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 01:24:09.5 |     -1.000 |     102290 |


|         2 | 01:24:09.5 |     -0.500 |     103081 |
|         3 | 01:24:09.5 |      0.000 |     103499 |
|         4 | 01:24:09.5 |      0.500 |     102799 |
|         5 | 01:24:09.5 |      1.000 |     102016 |


+-----------+------------+------------+------------+
generator scan ['d87feaf3'] (scan num: 3)





('d87feaf3-1509-42a4-91a5-82dd4f7cafa2',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/d87feaf3-1509-42a4-91a5-82dd4f7cafa2-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 01:24:09
Persistent Unique Scan ID: 'f79c5976-f976-454f-906c-9fdee6189ce9'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 01:24:10.0 |          -1 | 1104158696 |


|         2 | 01:24:10.1 |          -0 | 1097111679 |


|         3 | 01:24:10.2 |           0 | 1084649086 |
|         4 | 01:24:10.3 |           0 | 1078547304 |


|         5 | 01:24:10.4 |           1 | 1072685943 |


+-----------+------------+-------------+------------+
generator scan ['f79c5976'] (scan num: 4)





('f79c5976-f976-454f-906c-9fdee6189ce9',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 01:24:10
Persistent Unique Scan ID: '33c441b6-ed12-4c4e-b6bc-59be6f4fdcaa'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 01:24:10.9 | 1056474297 |


|         2 | 01:24:11.8 | 1077741315 |


|         3 | 01:24:12.8 | 1128944727 |


...

|        28 | 01:24:37.9 | 1104384879 |


|        29 | 01:24:38.9 | 1055051583 |


|        30 | 01:24:39.9 | 1081157198 |


+-----------+------------+------------+
generator count ['33c441b6'] (scan num: 5)





('33c441b6-ed12-4c4e-b6bc-59be6f4fdcaa',)